In [60]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import docx
import time

# 获取URL

In [61]:
url = 'https://www.xbiquge.cc/book/49549/35899003.html' #测试用 章节
#main_url =  'https://www.xbiquge.cc/book/49549/'       #豪婿
#main_url = 'https://www.xbiquge.cc/book/420/'           #龙王传说
main_url ='https://www.xbiquge.cc/book/46844/'          #终极斗罗
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0'
}

In [62]:
t = time.time()
while True:
    try:
        #加入timeout防止单次访问用时过长，timeout = 5防止对服务器造成过大负担
        main_response = requests.get(main_url, headers = header, timeout = 5)
        break
    except:
        print('---URL {} Failed...Try to connect again'.format(main_url))
print('---------Done!: used {:.2f}s--------'.format(time.time() - t))

---------Done!: used 2.38s--------


# 解码 乱码->中文

In [63]:
#www.xbiquge.cc 使用gbk进行编码，所以需要gbk解码
main_response.encoding = "gbk"
main_text = main_response.content.decode('gbk')

# 读入beautifulsoup进行解析

In [64]:
#html.parser是解析器，也可是lxml
main_soup = BeautifulSoup(main_text,'lxml')
#把整个网页转换成解析过的String，不过感觉不需要这一步
#直接使用main_text可以达到同样效果
strMain = main_soup.prettify()
#获得主标题
find_title = re.compile(r'<meta content="(.*?)" property="og:title"/>')
#获得作者
find_author = re.compile(r'<meta content="(.*?)" property="og:novel:author"/>')
mainTitle = find_title.findall(strMain)[0]
author = find_author.findall(strMain)[0]

# 获取小说list

In [65]:
#解析主网页的小说列表，笔趣阁都是以同一个形式加载的
pattern = re.compile(r'<a href="(\d+.html)">')
text_list = pattern.findall(main_soup.prettify())

In [66]:
#test block
text_list

['29074759.html',
 '29074760.html',
 '29074761.html',
 '29074762.html',
 '29074766.html',
 '29074767.html',
 '29074768.html',
 '29074769.html',
 '29074770.html',
 '29074771.html',
 '29074772.html',
 '29074773.html',
 '29074775.html',
 '29074776.html',
 '29074777.html',
 '29074778.html',
 '29074779.html',
 '29074780.html',
 '29074781.html',
 '29074782.html',
 '29074783.html',
 '29074785.html',
 '29074786.html',
 '29074787.html',
 '29074788.html',
 '29074789.html',
 '29074790.html',
 '29074791.html',
 '29074792.html',
 '29074793.html',
 '29074795.html',
 '29074796.html',
 '29074799.html',
 '29074801.html',
 '29074802.html',
 '29074803.html',
 '29074804.html',
 '29074805.html',
 '29074806.html',
 '29074807.html',
 '29074808.html',
 '29074810.html',
 '29074812.html',
 '29074813.html',
 '29074814.html',
 '29074815.html',
 '29074816.html',
 '29074817.html',
 '29074818.html',
 '29074819.html',
 '29074820.html',
 '29074821.html',
 '29074822.html',
 '29074823.html',
 '29074826.html',
 '29074829

# 每一个章节分别进行提取输出操作

   ## 1. 读取每一个章节的内容和标题

In [68]:
def write_to_doc(book):
    begin_time = time.time()
    chapter_url = main_url+book
    print('---Getting URL...---')
    while True:
        try:
            #加timeout防单词用时过长
            chapter_response = requests.get(chapter_url,headers = header,timeout = 5)
            break
        except:
            print('---URL {} Failed...Try to connect again'.format(book))
            continue
    print('-----URL {} used {:.2f}s-----'.format(book, time.time() - begin_time))
    #小说主体部分需要重新解码
    chapter_response.encoding = "gbk"
    chapter_text = chapter_response.content.decode('gbk')
    
    #以beautiful的形式读入，方便后续解析
    chapter_soup = BeautifulSoup(chapter_text,'lxml')

    #通过分析页面，笔趣阁小说主体部分在 id为content的div里
    div = chapter_soup.find_all('div',id='content')#,string='更多'
    #将获得的tag们全部变成string
    div_text =div[0].prettify()
    #网页中每个段落之间会有<br/>，刚好适合用来分段落
    content = div_text.split('<br/>')[1:]
    #再将过多的\n去除
    content = "".join(content).split('\n')[:-2]
    
    #获得章节名、文章名、以及来源
    title = chapter_soup.title.text
    mainTitle = title.split('_')[1]
    source = title.split('_')[2]
    chapterTitle = title.split('_')[0]
    
    print('---Fetched the contents---')
    
    return content, chapterTitle


## 2.写入word文档和同名txt文件

In [69]:
begin = time.time()
#新建一个docx文件
file=docx.Document()
#新建一个txt文件
file_handle = open('{}.txt'.format(mainTitle), mode = 'w')
#写入主标题和作者名
file.add_heading("{}".format(mainTitle),level=1)
file.add_heading("作者:{}".format(author),level=2)
file.add_page_break()
count = 0
for i in text_list:
    #count 用来计数，数字为章节数
    count  += 1
    print('-------------Doing chapter: {}------------'.format(count))
    
    content, chapterTitle = write_to_doc(i)
    #写入段落标题
    file.add_heading("{}".format(chapterTitle),level=2)
    
    print('----begin to write chapter {} into doc----'.format(count))
    #将爬取回的内容一段一段写入docx
    for i in content:
        if i !=  ' ':
            file.add_paragraph(i)
            file_handle.write(i)
            file_handle.write('\n')
    print('---Saving for chapter {}!---'.format(count))
    #加入分页，每一章的开头都是全新的一页纸
    file.add_page_break()
    #保存docx文件
    file.save("{}.docx".format(mainTitle)) #输出的word名字
    #sleep(1)防止对服务器造成过大负担
    time.sleep(1)
    print('---Done for chapter: {}---\n'.format(count))
file_handle.close()
print('--------------------------------DONE!!!!!!!-------------------------')
print('------------------uesd: {:.2f}s-------------------'.format(time.time() - begin))

-------------Doing chapter: 1------------
---Getting URL...---
-----URL 29074759.html used 3.63s-----
---Fetched the contents---
----begin to write chapter 1 into doc----
---Saving for chapter 1!---
---Done for chapter: 1---

-------------Doing chapter: 2------------
---Getting URL...---
---URL 29074760.html Failed...Try to connect again
-----URL 29074760.html used 23.56s-----
---Fetched the contents---
----begin to write chapter 2 into doc----
---Saving for chapter 2!---
---Done for chapter: 2---

-------------Doing chapter: 3------------
---Getting URL...---
-----URL 29074761.html used 5.34s-----
---Fetched the contents---
----begin to write chapter 3 into doc----
---Saving for chapter 3!---
---Done for chapter: 3---

-------------Doing chapter: 4------------
---Getting URL...---
-----URL 29074762.html used 9.94s-----
---Fetched the contents---
----begin to write chapter 4 into doc----
---Saving for chapter 4!---
---Done for chapter: 4---

-------------Doing chapter: 5------------
--

-----URL 29074802.html used 1.00s-----
---Fetched the contents---
----begin to write chapter 35 into doc----
---Saving for chapter 35!---
---Done for chapter: 35---

-------------Doing chapter: 36------------
---Getting URL...---
-----URL 29074803.html used 0.99s-----
---Fetched the contents---
----begin to write chapter 36 into doc----
---Saving for chapter 36!---
---Done for chapter: 36---

-------------Doing chapter: 37------------
---Getting URL...---
-----URL 29074804.html used 2.58s-----
---Fetched the contents---
----begin to write chapter 37 into doc----
---Saving for chapter 37!---
---Done for chapter: 37---

-------------Doing chapter: 38------------
---Getting URL...---
-----URL 29074805.html used 0.98s-----
---Fetched the contents---
----begin to write chapter 38 into doc----
---Saving for chapter 38!---
---Done for chapter: 38---

-------------Doing chapter: 39------------
---Getting URL...---
---URL 29074806.html Failed...Try to connect again
---URL 29074806.html Failed..

-----URL 29093694.html used 4.00s-----
---Fetched the contents---
----begin to write chapter 69 into doc----
---Saving for chapter 69!---
---Done for chapter: 69---

-------------Doing chapter: 70------------
---Getting URL...---
-----URL 29096423.html used 6.89s-----
---Fetched the contents---
----begin to write chapter 70 into doc----
---Saving for chapter 70!---
---Done for chapter: 70---

-------------Doing chapter: 71------------
---Getting URL...---
-----URL 29110045.html used 1.98s-----
---Fetched the contents---
----begin to write chapter 71 into doc----
---Saving for chapter 71!---
---Done for chapter: 71---

-------------Doing chapter: 72------------
---Getting URL...---
-----URL 29113336.html used 4.44s-----
---Fetched the contents---
----begin to write chapter 72 into doc----
---Saving for chapter 72!---
---Done for chapter: 72---

-------------Doing chapter: 73------------
---Getting URL...---
-----URL 29141286.html used 5.61s-----
---Fetched the contents---
----begin to w

---Done for chapter: 103---

-------------Doing chapter: 104------------
---Getting URL...---
-----URL 29450948.html used 3.24s-----
---Fetched the contents---
----begin to write chapter 104 into doc----
---Saving for chapter 104!---
---Done for chapter: 104---

-------------Doing chapter: 105------------
---Getting URL...---
---URL 29453271.html Failed...Try to connect again
-----URL 29453271.html used 13.43s-----
---Fetched the contents---
----begin to write chapter 105 into doc----
---Saving for chapter 105!---
---Done for chapter: 105---

-------------Doing chapter: 106------------
---Getting URL...---
-----URL 29463434.html used 2.07s-----
---Fetched the contents---
----begin to write chapter 106 into doc----
---Saving for chapter 106!---
---Done for chapter: 106---

-------------Doing chapter: 107------------
---Getting URL...---
-----URL 29466778.html used 4.36s-----
---Fetched the contents---
----begin to write chapter 107 into doc----
---Saving for chapter 107!---
---Done for 

-----URL 29636923.html used 3.93s-----
---Fetched the contents---
----begin to write chapter 138 into doc----
---Saving for chapter 138!---
---Done for chapter: 138---

-------------Doing chapter: 139------------
---Getting URL...---
-----URL 29639891.html used 3.78s-----
---Fetched the contents---
----begin to write chapter 139 into doc----
---Saving for chapter 139!---
---Done for chapter: 139---

-------------Doing chapter: 140------------
---Getting URL...---
-----URL 29648800.html used 2.68s-----
---Fetched the contents---
----begin to write chapter 140 into doc----
---Saving for chapter 140!---
---Done for chapter: 140---

-------------Doing chapter: 141------------
---Getting URL...---
-----URL 29652196.html used 2.94s-----
---Fetched the contents---
----begin to write chapter 141 into doc----
---Saving for chapter 141!---
---Done for chapter: 141---

-------------Doing chapter: 142------------
---Getting URL...---
-----URL 29664934.html used 3.34s-----
---Fetched the contents--

...

-----URL 35655491.html used 4.03s-----
---Fetched the contents---
----begin to write chapter 1062 into doc----
---Saving for chapter 1062!---
---Done for chapter: 1062---

-------------Doing chapter: 1063------------
---Getting URL...---
-----URL 35659345.html used 1.79s-----
---Fetched the contents---
----begin to write chapter 1063 into doc----
---Saving for chapter 1063!---
---Done for chapter: 1063---

-------------Doing chapter: 1064------------
---Getting URL...---
-----URL 35660952.html used 6.14s-----
---Fetched the contents---
----begin to write chapter 1064 into doc----
---Saving for chapter 1064!---
---Done for chapter: 1064---

-------------Doing chapter: 1065------------
---Getting URL...---
-----URL 35663961.html used 1.00s-----
---Fetched the contents---
----begin to write chapter 1065 into doc----
---Saving for chapter 1065!---
---Done for chapter: 1065---

-------------Doing chapter: 1066------------
---Getting URL...---
-----URL 35665624.html used 10.37s-----
---Fetch

---Saving for chapter 1096!---
---Done for chapter: 1096---

-------------Doing chapter: 1097------------
---Getting URL...---
-----URL 35746645.html used 1.48s-----
---Fetched the contents---
----begin to write chapter 1097 into doc----
---Saving for chapter 1097!---
---Done for chapter: 1097---

-------------Doing chapter: 1098------------
---Getting URL...---
-----URL 35749450.html used 3.26s-----
---Fetched the contents---
----begin to write chapter 1098 into doc----
---Saving for chapter 1098!---
---Done for chapter: 1098---

-------------Doing chapter: 1099------------
---Getting URL...---
-----URL 35750903.html used 1.67s-----
---Fetched the contents---
----begin to write chapter 1099 into doc----
---Saving for chapter 1099!---
---Done for chapter: 1099---

-------------Doing chapter: 1100------------
---Getting URL...---
-----URL 35753966.html used 0.89s-----
---Fetched the contents---
----begin to write chapter 1100 into doc----
---Saving for chapter 1100!---
---Done for chapt

---Done for chapter: 1130---

-------------Doing chapter: 1131------------
---Getting URL...---
-----URL 35843804.html used 8.47s-----
---Fetched the contents---
----begin to write chapter 1131 into doc----
---Saving for chapter 1131!---
---Done for chapter: 1131---

-------------Doing chapter: 1132------------
---Getting URL...---
-----URL 35845239.html used 0.91s-----
---Fetched the contents---
----begin to write chapter 1132 into doc----
---Saving for chapter 1132!---
---Done for chapter: 1132---

-------------Doing chapter: 1133------------
---Getting URL...---
-----URL 35847498.html used 6.20s-----
---Fetched the contents---
----begin to write chapter 1133 into doc----
---Saving for chapter 1133!---
---Done for chapter: 1133---

-------------Doing chapter: 1134------------
---Getting URL...---
-----URL 35848797.html used 3.43s-----
---Fetched the contents---
----begin to write chapter 1134 into doc----
---Saving for chapter 1134!---
---Done for chapter: 1134---

-------------Doing

---Saving for chapter 1164!---
---Done for chapter: 1164---

-------------Doing chapter: 1165------------
---Getting URL...---
-----URL 35956972.html used 0.84s-----
---Fetched the contents---
----begin to write chapter 1165 into doc----
---Saving for chapter 1165!---
---Done for chapter: 1165---

-------------Doing chapter: 1166------------
---Getting URL...---
-----URL 35958347.html used 1.13s-----
---Fetched the contents---
----begin to write chapter 1166 into doc----
---Saving for chapter 1166!---
---Done for chapter: 1166---

-------------Doing chapter: 1167------------
---Getting URL...---
-----URL 35960661.html used 3.67s-----
---Fetched the contents---
----begin to write chapter 1167 into doc----
---Saving for chapter 1167!---
---Done for chapter: 1167---

-------------Doing chapter: 1168------------
---Getting URL...---
-----URL 35961954.html used 3.76s-----
---Fetched the contents---
----begin to write chapter 1168 into doc----
---Saving for chapter 1168!---
---Done for chapt

---Saving for chapter 1198!---
---Done for chapter: 1198---

-------------Doing chapter: 1199------------
---Getting URL...---
-----URL 36034409.html used 4.33s-----
---Fetched the contents---
----begin to write chapter 1199 into doc----
---Saving for chapter 1199!---
---Done for chapter: 1199---

-------------Doing chapter: 1200------------
---Getting URL...---
-----URL 36036550.html used 3.01s-----
---Fetched the contents---
----begin to write chapter 1200 into doc----
---Saving for chapter 1200!---
---Done for chapter: 1200---

-------------Doing chapter: 1201------------
---Getting URL...---
-----URL 36038708.html used 1.88s-----
---Fetched the contents---
----begin to write chapter 1201 into doc----
---Saving for chapter 1201!---
---Done for chapter: 1201---

-------------Doing chapter: 1202------------
---Getting URL...---
-----URL 36039980.html used 0.89s-----
---Fetched the contents---
----begin to write chapter 1202 into doc----
---Saving for chapter 1202!---
---Done for chapt

In [70]:
print(15127.34/60//60)
print(1512.34/60%60)

4.0
25.205666666666666


In [76]:
file_handle = open('/test/test.txt', mode = 'w')
file_handle.write('Hello World')
file_handle.write('\n')
file_handle.write('Hello World')
file_handle.close()

FileNotFoundError: [Errno 2] No such file or directory: '/test/test.txt'